In [1]:
%tensorflow_version 1.x
#No need to install you can just specify version
#pip install tensorflow==1.15

TensorFlow 1.x selected.


In [2]:
import os
os.chdir('/content/drive/My Drive/project/ml_models')

In [3]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report
df_train = pd.read_csv('train.tsv',sep='\t')

In [4]:
df_test = pd.read_csv('dev.tsv',sep='\t')

In [5]:
df_train.head()

,Unnamed: 0,ids,body,Uncertainity of Post_Diagnosis,Results and Side-Effects Observed,Medical Assistance,Diet and Maintenance,Information Source,Concepts
0,0,LIPITOR.449.txt,Extreme tiredness and flatulence. Not sure whe...,0,1,0,0,0,flatulence|exhaustion|tired|Not sure|Extreme|t...
1,1,LIPITOR.188.txt,1/7/05-continued. not all of it posted before....,0,1,0,1,0,package insert|depression|Lipitor|cholesterol|...
2,2,LIPITOR.541.txt,So sad to see so many with problems like mine!...,1,1,1,0,0,muscle pain|joint pain|depression|Lipitor|Lipi...
3,3,LIPITOR.810.txt,Within 1 month time developed severe depressio...,0,1,0,0,0,severe depression|headaches|Lipitor|statins|Li...
4,4,LIPITOR.393.txt,I have been on lipitor for 10 years for heart ...,0,1,0,0,0,leg weakness|changed|experience|Potassium|cram...


In [6]:
df_test.head()

,Unnamed: 0,ids,body,Uncertainity of Post_Diagnosis,Results and Side-Effects Observed,Medical Assistance,Diet and Maintenance,Information Source,Concepts
0,0,LIPITOR.595.txt,"Swelling left arm, very severe itching, intole...",0,1,0,0,0,Swelling|itching|left arm|hand|dreams|bruise|v...
1,1,ARTHROTEC.101.txt,"1st pill taken with food, a few hours after i ...",0,1,0,0,0,side effects|depression|cramping|upset|experie...
2,2,LIPITOR.367.txt,episode of intense dizziness lasting nearly an...,0,1,0,0,0,dizziness|lassitude|chills|shivers|problem|wor...
3,3,LIPITOR.74.txt,After taking Crestor and having muscle pain an...,0,1,0,0,1,liver problems|decreased|itchy|control|crawly|...
4,4,LIPITOR.389.txt,"75 yo mother-in-law has memory loss, hair loss...",0,1,0,1,0,lack of appetite|hair loss|stroke|sciatica|los...


In [7]:
train_body =df_train['body'].to_list()
test_body =df_test['body'].to_list()

In [8]:
import re
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

for idx,text in enumerate(train_body):
  train_body[idx] = clean_text(text)

for idx,text in enumerate(test_body):
  test_body[idx] = clean_text(text)

In [9]:
from collections import Counter

str_total = ' '.join(train_body) + ' '.join(test_body)
list_total = str_total.split()
dict_count = Counter(list_total)

vocab = dict()
#vocab['unk']=0
for key,val in dict_count.items():
  vocab[key] = len(vocab)

In [10]:
print(len(vocab))
vocab

6640


{'extreme': 0,
 'tiredness': 1,
 'and': 2,
 'flatulence': 3,
 'not': 4,
 'sure': 5,
 'whether': 6,
 'it': 7,
 'is': 8,
 'the': 9,
 'lipitor': 10,
 'but': 11,
 'i': 12,
 'am': 13,
 'now': 14,
 'tired': 15,
 'to': 16,
 'point': 17,
 'of': 18,
 'exhaustion': 19,
 'work': 20,
 'me': 21,
 'getting': 22,
 'older': 23,
 'or': 24,
 '1': 25,
 '7': 26,
 '05': 27,
 'continued': 28,
 'all': 29,
 'posted': 30,
 'before': 31,
 'here': 32,
 'continuation': 33,
 'never': 34,
 'filled': 35,
 'as': 36,
 'my': 37,
 'instincts': 38,
 'told': 39,
 'was': 40,
 'something': 41,
 'more': 42,
 'generally': 43,
 'a': 44,
 'well': 45,
 'balanced': 46,
 'type': 47,
 'person': 48,
 'depression': 49,
 'did': 50,
 'come': 51,
 'on': 52,
 'rather': 53,
 'suddenly': 54,
 'about': 55,
 'same': 56,
 'time': 57,
 'when': 58,
 'started': 59,
 'advice': 60,
 'you': 61,
 'would': 62,
 'be': 63,
 'listen': 64,
 'your': 65,
 'body': 66,
 'may': 67,
 'trying': 68,
 'tell': 69,
 'try': 70,
 'diet': 71,
 'excercise': 72,
 'lower

In [11]:
x_train = list()
x_test = list()
def encode_text(toks,type='train'):
  encod_arr = [None]*len(toks)
  try:
    for idx,tok in enumerate(toks):
      encod_arr[idx] = vocab[tok]
  except: pass

  if type =='train' : x_train.append(encod_arr)
  if type =='test' : x_test.append(encod_arr)


for item in train_body:
  encode_text(item.split(), 'train')

for item in test_body:
  encode_text(item.split(), 'test')

In [12]:
print(x_train[0],train_body[0])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 9, 17, 18, 19, 8, 7, 20, 8, 7, 21, 22, 23, 24, 8, 7, 9, 10] extreme tiredness and flatulence not sure whether it is the lipitor but i am now tired to the point of exhaustion is it work is it me getting older or is it the lipitor


In [13]:
from keras import backend as K


def k_dot(vec_x, vec_y):
    """Dot operation for Keras compatible for theano and tensorflow.
    Args:
        vec_x: vector.
        vec_y: vector.
    Returns:
        A dot product operation by two vectors.
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(vec_x, K.expand_dims(vec_y)), axis=-1)
    else:
        return K.dot(vec_x, vec_y)

Using TensorFlow backend.


In [14]:
import keras
from keras import initializers, regularizers, constraints
from keras.engine.topology import Layer


class Attention(Layer):
    def __init__(self, bias=True, **kwargs):
        """The implementation of an attention mechanism layer.
        Args:
            bias: boolean, adapt a bias modification or not.
            kwargs: any keyword arguments that base layer accepts.
        """
        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        """Create a trainable weight variable.
        This attention layer accepts the tensor that has three
        dimensions: (batch_size, time_steps, input_dim).
        Args:
            input_shape: 3d Input, (batch_size, time_steps, input_dim).
        Raises:
            ValueError: if input_shape is not 3 dimensions.
        """
        assert len(input_shape) == 3

        self.W = self.add_weight(
            name='W', shape=(input_shape[-1], input_shape[-1],),
            initializer=keras.initializers.get('uniform')
        )

        self.b = self.add_weight(
            name='bias', shape=(input_shape[-1],),
            initializer='zero'
        )

        self.u = self.add_weight(
            name='context_vector', shape=(input_shape[-1],),
            initializer=keras.initializers.get('uniform')
        )

        super(Attention, self).build(input_shape)

    def call(self, x):

        att_weights = self._get_attention_weights(x)

        # Reshape the attention weights to match the dimensions of X
        att_weights = K.expand_dims(att_weights)
        # Multiply each input by its attention weights
        # TODO: use backend function for multiply
        weighted_input = keras.layers.Multiply()([x, att_weights])

        # Sum in the direction of the time-axis.
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

    def _get_attention_weights(self, x):
        """Calculate attention weights.
        Args:
            x: Input array.
        Returns:
            a: attention weights.
        """
        u_xw = k_dot(x, self.W)
        if self.bias:
            u_xw += self.b
        u_tw = K.tanh(u_xw)
        a_tw = k_dot(u_tw, self.u)

        # apply softmax to get probability for attention
        a = K.softmax(a_tw)

        return a

In [15]:
from keras import Input, Model
from keras.layers import Embedding, Dense, Bidirectional, CuDNNLSTM, TimeDistributed,Lambda

max_features = len(vocab)
maxlen_sentence = 16
maxlen_word = 25
batch_size = 32
embedding_dims = 50
epochs = 10

class HAN():
    def __init__(self, maxlen_sentence=16, maxlen_word=25, max_features=len(vocab), embedding_dims=50,
                 class_num=2,
                 last_activation='softmax'):
        self.maxlen_sentence = maxlen_sentence
        self.maxlen_word = maxlen_word
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation
        #self.embedding_matrix = embedding_matrix


    def build_word_encoder(self):
        # Word part
        input_word = Input(shape=(self.maxlen_word,))
        embedded_word = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen_word)(input_word)
        x_word = Bidirectional(CuDNNLSTM(128, return_sequences=True))(embedded_word)  # LSTM or GRU
        word_att = Attention(self.maxlen_word,name='word_attention')(x_word)
        model_word = Model(input_word, word_att)
        return model_word

    def build_sent_encoder(self, sent_encoder):
        # Sentence part
        sent_lstm = Bidirectional(CuDNNLSTM(128, return_sequences=True))(sent_encoder)  # LSTM or GRU

        sent_att = Attention(name='sent_attention')(sent_lstm)
        return sent_att
      
    def get_model(self):
        text_input = Input(shape=(self.maxlen_sentence, self.maxlen_word))
        # encode sentences into a single vector per sentence
        self.model_word = self.build_word_encoder()
        # time distribute word model to accept text input
        sent_encoder = TimeDistributed(self.model_word)(text_input)

        sent_att = self.build_sent_encoder(sent_encoder)
        # dense the output to 2 because the result is a binary classification.
        output_tensor = Dense(2, activation='softmax', name='classification')(sent_att)
        # Create Sentence-level Model
        self.model = Model(text_input, output_tensor)
        return Model(text_input, output_tensor)

    def show_word_attention(self, x):
        """Show the prediction of the word level attention.

        Args:
            x: the input array with size of (max_sent_length,).

        Returns:
            Attention weights.
        """
        att_layer = self.model_word.get_layer('word_attention')
        prev_tensor = att_layer.input

        # Create a temporary dummy layer to hold the
        # attention weights tensor
        dummy_layer = Lambda(
            lambda x: att_layer._get_attention_weights(x)
        )(prev_tensor)

        return Model(self.model_word.input, dummy_layer).predict(x)

    def show_sent_attention(self, x):
        """Show the prediction of the sentence level attention.

        Args:
            x: the input array with the size of (max_sent_num, max_sent_length).

        Returns:
            Attention weights.
        """
        att_layer = self.model.get_layer('sent_attention')
        prev_tensor = att_layer.input

        dummy_layer = Lambda(
            lambda x: att_layer._get_attention_weights(x)
        )(prev_tensor)

        return Model(self.model.input, dummy_layer).predict(x)
      
    

In [16]:
def word_attention_to_df(sent_tokenized_review, word_att):
      

      ori_sents = sent_tokenized_review
      # split sentences into words
      ori_words = [x for x in ori_sents.split()]
      # truncate attentions to have equal size of number of words per sentence
      ln=len(ori_words)
      f=ln%maxlen_word
      f1=ln//maxlen_word  
      
      words_attn=[]
      for i in word_att[::-1]:
        l=min(len(i),ln)
        words_attn.extend(i[::-1][:l])
        ln=ln-l
        if ln<=0:
          break

      words_attn.reverse()    
        
      f1=f1+1
      if f==0:
        f=maxlen_word
        f1=f1-1
      mylist1=[]
      mylist2=[]
      dict_attn=dict(zip(ori_words,words_attn))
      dict_items = dict_attn.items()       
      k1=0
      for i in range(f1):
        mylist1.append(dict(zip(ori_words[k1:f+k1],words_attn[k1:k1+f])))
        listToStr = ' '.join(map(str, ori_words[k1:k1+f]))         
        mylist2.append(listToStr)
        k1=f
        f=f+maxlen_word 
        

      return pd.DataFrame(zip(mylist1,mylist2),
                  columns=['word_att', 'sentence'])


In [17]:
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.utils import to_categorical


print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen_sentence * maxlen_word)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen_sentence * maxlen_word)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen_sentence * maxlen_word)
x_train = x_train.reshape((len(x_train), maxlen_sentence, maxlen_word))
x_test = x_test.reshape((len(x_test), maxlen_sentence, maxlen_word))
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)


han=HAN()
print('Build model...')
model = han.get_model()
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
categories = ['Uncertainity of Post_Diagnosis', 'Results and Side-Effects Observed', 'Medical Assistance', 'Diet and Maintenance', 'Information Source']
id_num=4
show_results=[]
for category in categories:
    print('Reporting Category: ', category)

    y_train_label = df_train[category].to_list()
    y_test_label = df_test[category].to_list()

    y_train = to_categorical(y_train_label, num_classes=None)
    y_test = to_categorical(y_test_label, num_classes=None)

    model.fit(x_train,y_train,
              batch_size=batch_size,
              epochs=epochs,
              callbacks=[early_stopping],
              validation_data=(x_test, y_test))
    print('Test...')
    result = model.predict(x_test)
    predictions = [np.argmax(x) for x in result]
    print(classification_report(y_test_label,predictions))
    
    df=word_attention_to_df(train_body[id_num],han.show_word_attention(x_train[id_num]))
    show_results.append(df)
    #print(train_body[id_num],"\n")
    #print(df) 


Pad sequences (samples x time)...
x_train shape: (942, 16, 25)
x_test shape: (300, 16, 25)
Build model...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train...
Reporting Category:  Uncertainity of Post_Diagnosis

Train on 942 samples, validate on 300 samples
Epoch 1/10
942/942 [==============================] - 3s 3ms/step - loss: 0.4833 - accuracy: 0.8227 - val_loss: 0.3973 - val_accuracy: 0.8567
Epoch 2/10
128/942 [===>..........................] - ETA: 1s - loss: 0.4554 - accuracy: 0.8125

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


942/942 [==============================] - 2s 2ms/step - loss: 0.4401 - accuracy: 0.8217 - val_loss: 0.3814 - val_accuracy: 0.8567
Epoch 3/10
942/942 [==============================] - 2s 2ms/step - loss: 0.3605 - accuracy: 0.8482 - val_loss: 0.3620 - val_accuracy: 0.8633
Epoch 4/10
942/942 [==============================] - 2s 2ms/step - loss: 0.1856 - accuracy: 0.9289 - val_loss: 0.4243 - val_accuracy: 0.8167
Epoch 5/10
942/942 [==============================] - 2s 2ms/step - loss: 0.0716 - accuracy: 0.9798 - val_loss: 0.5436 - val_accuracy: 0.8500
Epoch 6/10
942/942 [==============================] - 2s 2ms/step - loss: 0.0207 - accuracy: 0.9968 - val_loss: 0.9527 - val_accuracy: 0.7533
Epoch 7/10
942/942 [==============================] - 2s 2ms/step - loss: 0.0153 - accuracy: 0.9968 - val_loss: 0.7721 - val_accuracy: 0.8333
Epoch 8/10
942/942 [==============================] - 2s 2ms/step - loss: 0.0245 - accuracy: 0.9979 - val_loss: 0.8578 - val_accuracy: 0.7933
Epoch 9/10
942/94

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Reporting Category:  Medical Assistance
Train on 942 samples, validate on 300 samples
Epoch 1/10
942/942 [==============================] - 2s 2ms/step - loss: 0.8312 - accuracy: 0.7197 - val_loss: 0.5623 - val_accuracy: 0.8100
Epoch 2/10
942/942 [==============================] - 2s 2ms/step - loss: 0.5242 - accuracy: 0.7877 - val_loss: 0.4871 - val_accuracy: 0.8100
Epoch 3/10
942/942 [==============================] - 2s 2ms/step - loss: 0.5213 - accuracy: 0.7877 - val_loss: 0.4879 - val_accuracy: 0.8100
Epoch 4/10
942/942 [==============================] - 2s 2ms/step - loss: 0.5185 - accuracy: 0.7877 - val_loss: 0.4856 - val_accuracy: 0.8100
Epoch 5/10
942/942 [==============================] - 2s 2ms/step - loss: 0.5153 - accuracy: 0.7877 - val_loss: 0.4882 - val_accuracy: 0.8100
Epoch 6/10
942/942 [==============================] - 2s 2ms/step - loss: 0.5126 - accuracy: 0.7877 - val_loss: 0.4811 - val_accuracy: 0.8100
Epoch 7/10
942/942 [==============================] - 2s 2ms/s

In [18]:
print(train_body[id_num],"\n")
for i in range(len(show_results)):
  print(categories[i])
  print(show_results[i])
  print('\n\n')

i have been on lipitor for 10 years for heart maintenance 20mg iwas changed to 10mg and added tricor 145 mg because of high ldl i do experience leg weakness and cramping cramping relieved with potassium 

Uncertainity of Post_Diagnosis
                                            word_att                                               word
0  {'i': 0.043330774, 'have': 0.0440861, 'been': ...  i have been on lipitor for 10 years for heart ...
1  {'20mg': 0.023785641, 'iwas': 0.02562928, 'cha...  20mg iwas changed to 10mg and added tricor 145...



Results and Side-Effects Observed
                                            word_att                                               word
0  {'i': 0.0038137329, 'have': 0.001874147, 'been...  i have been on lipitor for 10 years for heart ...
1  {'20mg': 0.35244894, 'iwas': 0.237391, 'change...  20mg iwas changed to 10mg and added tricor 145...



Medical Assistance
                                            word_att                             

#dummy

In [204]:
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.utils import to_categorical


print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen_sentence * maxlen_word)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen_sentence * maxlen_word)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen_sentence * maxlen_word)
x_train = x_train.reshape((len(x_train), maxlen_sentence, maxlen_word))
x_test = x_test.reshape((len(x_test), maxlen_sentence, maxlen_word))
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
#categories = ['Uncertainity of Post_Diagnosis', 'Results and Side-Effects Observed', 'Medical Assistance', 'Diet and Maintenance', 'Information Source']
category='Uncertainity of Post_Diagnosis'
y_train_label = df_train[category].to_list()
y_test_label = df_test[category].to_list()
y_train = to_categorical(y_train_label, num_classes=None)
y_test = to_categorical(y_test_label, num_classes=None)
han=HAN()
print('Build model...')
model = han.get_model()
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
model.fit(x_train,y_train,
              batch_size=batch_size,
              epochs=epochs,
              callbacks=[early_stopping],
              validation_data=(x_test, y_test))
print('Test...')
result = model.predict(x_test)
predictions = [np.argmax(x) for x in result]
print(classification_report(y_test_label,predictions))
"""for category in categories:
    print('Reporting Category: ', category)

    y_train_label = df_train[category].to_list()
    y_test_label = df_test[category].to_list()

    y_train = tf.keras.utils.to_categorical(y_train_label, num_classes=None)
    y_test = tf.keras.utils.to_categorical(y_test_label, num_classes=None)

    model.fit(x_train,y_train,
              batch_size=batch_size,
              epochs=epochs,
              callbacks=[early_stopping],
              validation_data=(x_test, y_test))
    print('Test...')
    result = model.predict(x_test)
    predictions = [np.argmax(x) for x in result]
    print(classification_report(y_test_label,predictions))"""

Pad sequences (samples x time)...
x_train shape: (942, 16, 25)
x_test shape: (300, 16, 25)
Build model...
Train...
Train on 942 samples, validate on 300 samples
Epoch 1/10
864/942 [==========================>...] - ETA: 0s - loss: 0.4979 - accuracy: 0.8009

KeyboardInterrupt: ignored

In [120]:
x_train[0]

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

In [121]:
hn=han.show_word_attention(x_train[0])


In [186]:
def word_attention_to_df(sent_tokenized_review, word_att):             
        ori_sents = sent_tokenized_review
        # split sentences into words
        ori_words = [x for x in ori_sents.split()]
        # truncate attentions to have equal size of number of words per sentence
        ln=len(ori_words)
        f=ln%maxlen_word
        f1=ln//maxlen_word  
        
        words_attn=[]
        for i in word_att[::-1]:
          l=min(len(i),ln)
          words_attn.extend(i[::-1][:l])
          ln=ln-l
          if ln<=0:
            break
  
        words_attn.reverse()    
          
        f1=f1+1
        if f==0:
          f=maxlen_word
          f1=f1-1
        mylist1=[]
        mylist2=[]
        dict_attn=dict(zip(ori_words,words_attn))
        dict_items = dict_attn.items()       
        k1=0
        for i in range(f1):
          mylist1.append(dict(zip(ori_words[k1:f+k1],words_attn[k1:k1+f])))
          listToStr = ' '.join(map(str, ori_words[k1:k1+f]))         
          mylist2.append(listToStr)
          k1=f
          f=f+maxlen_word 
         

        return pd.DataFrame(zip(mylist1,mylist2),
                    columns=['word_att', 'word'])

In [187]:
def show_word_attention(id):
  df=word_attention_to_df(train_body[id],han.show_word_attention(x_train[id]))
  print(train_body[id],"\n")
  print(df)  

In [188]:
id_num=4
show_word_attention(id_num)

i have been on lipitor for 10 years for heart maintenance 20mg iwas changed to 10mg and added tricor 145 mg because of high ldl i do experience leg weakness and cramping cramping relieved with potassium 

                                            word_att                                               word
0  {'i': 0.040872823, 'have': 0.040732674, 'been'...  i have been on lipitor for 10 years for heart ...
1  {'20mg': 0.034686994, 'iwas': 0.035907034, 'ch...  20mg iwas changed to 10mg and added tricor 145...


In [135]:
len(train_body[8].split())

164